In [225]:
import sqlite3
import pandas as pd
import re
from razdel import sentenize

In [226]:
df = pd.read_csv("tachka.csv")

### Создаем базу данных

In [255]:
con = sqlite3.connect('tachka_database.db') 
cur = con.cursor()

Таблица с ссылками и их id

In [256]:
cur.execute("""
CREATE TABLE urls (
    url_id INT, 
    url TEXT, 
    PRIMARY KEY (url_id)
)
""")
#таблица с авторами и их id
cur.execute("""
CREATE TABLE authors (
    author_id INT, 
    author TEXT, 
    PRIMARY KEY (author_id)
)
""")
#таблица с номером предложения, id источника и автора
cur.execute("""
CREATE TABLE sentences_table ( 
    author_id INT, 
    url_id INT,
    sentence_id INT,
    sentence TEXT,
    PRIMARY KEY (sentence_id)
)
""")
#таблица со словами
cur.execute("""
CREATE TABLE words_table (
    sentence_id INT, 
    word TEXT, 
    lemma TEXT,
    pos TEXT
)
""")

con.commit()

Заполняем таблицы

In [257]:
urls_dict = {}
i = 1
for url in df['Url']:
    urls_dict[url] = i
    cur.execute("INSERT INTO urls VALUES(?, ?);", (i, url))
    i+=1
    con.commit()

In [258]:
authors_dict = {}
i = 1
for author in set(df['Author']):
    authors_dict[author] = i
    cur.execute("INSERT INTO authors VALUES(?, ?);", (i, author))
    i+=1
    con.commit()

## Обрабатываем тексты для более удобного разделения на предложения и добавляем таблицу с предложениями в бд

Поскольку это соцсеть, со знаками препинания здесь все плохо: некоторые предложения без точки, но заканчиваются переносом строки, некоторые предложения заканчиваются скобочками.

In [259]:
def text_processing(text):
    #обработка текста для более удобного разделения на предложения
    text = re.sub('\n+', '\n', text)
    text = re.sub(r'\n⠀\n', r'\n', text)
    text = re.sub(r'(\w)(\n)', r'\1.\2', text)
    text = re.sub(r';\n', r'.\n', text)
    text = re.sub(r'\n ?\d[).]', '', text)
    text = re.sub(r'(\d.)([А-ЯЁ])', r'\2', text)
    text = re.sub(r'(\w)([()]+)(\s+)([А-ЯЁ\d])', r'\1\2.\3\4', text)
    text = re.sub(r'(\w)(-)', r'\1 \2', text)
    text = re.sub(r'(\d+)(й|ый|го|ого|ому|му|у|м|ым|ом)', r'\1-\2', text)
    for elem in re.findall(r':\s+[А-ЯЁ]', text):
        text = text.replace(elem, elem.lower())
    text = re.sub('(\n)(\.{3})', r'\2', text)
    text = re.sub('_+', '', text)
    text = re.sub(r'(:)(⠀*\n)([а-яё])', r'\1 \3', text)
    text = re.sub('\n\.', '\n', text)
    text = re.sub(r'(.)([А-ЯЁ])',r'\1 \2', text)
    return text

In [260]:
sentence_id = 1
for index, row in df.iterrows():
    url_id = urls_dict[row['Url']]
    author_id = authors_dict[row['Author']]
    text = text_processing (row['Text'])
    for sentence in list(sentenize(text)):
        sentence_tuple = (author_id, url_id, sentence_id, sentence.text)
        cur.execute("INSERT INTO sentences_table VALUES(?, ?, ?, ?);", sentence_tuple)
        sentence_id += 1
        con.commit()

In [261]:
df_try = pd.read_sql_query('SELECT * FROM sentences_table', con=con)
df_try

,author_id,url_id,sentence_id,sentence
0,97,1,1,"Что видят люди, глядя на эвок 1 поколения?"
1,97,1,2,"Красивый, стильный, сравнительно недорогой кро..."
2,97,1,3,"Что видит фанат марки, ездивший на нормальных ..."
3,97,1,4,"Всратый шмат чистого убожества, беззастенчиво ..."
4,97,1,5,Первое поколение эвока - это кровавый выпердыш...
...,...,...,...,...
5468,92,171,5469,"Отличный байк для начинающих, но ценник как на..."
5469,19,172,5470,"Всем вечер добрый,хочу рассказать за эксплуата..."
5470,19,172,5471,Данный аппарат на 1.6t cvt 4wd 190 японских ко...
5471,19,172,5472,"И так,начнем с того что это мой первый автомоб..."


## Морфологическая обработка предложений

In [262]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    PER,
    Doc
)

In [263]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [264]:
sentences = """
SELECT sentence_id, sentence
FROM sentences_table
"""
cur.execute(sentences)
data = cur.fetchall()

In [265]:
for elem in data:
    sentence_id = elem[0]
    doc = Doc(elem[1])
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if token.pos != 'PUNCT':
            word = token.text.lower()
            lemma = token.lemma
            POS = token.pos  
            word_tuple = (sentence_id, word, lemma, POS)
            cur.execute("INSERT INTO words_table VALUES(?, ?, ?, ?);", word_tuple)
            con.commit()

In [266]:
df_words = pd.read_sql_query('SELECT * FROM words_table', con=con)
df_words

,sentence_id,word,lemma,pos
0,1,что,что,PRON
1,1,видят,видеть,VERB
2,1,люди,человек,NOUN
3,1,глядя,глядеть,VERB
4,1,на,на,ADP
...,...,...,...,...
80846,5473,мой,мой,DET
80847,5473,отзыв,отзыв,NOUN
80848,5473,будет,быть,AUX
80849,5473,кому,кто,PRON
